# Node undersampling

1. Truncate nodes to 5 decimals
2. Find duplicates among nodes
3. Keep only the first node in both nodes and roads
4. Scan all roads to remove the duplicated nodes

Note: con 2 cifre decimali la risoluzione è di circa 2km... bene ma non benissimo

In [1]:
import json

In [2]:
nodes = json.load(open('nodes.json', 'r'))
roads = json.load(open('roads.json', 'r'))

In [3]:
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [41]:
#not really general, I make it work here for my points
def truncate(num):
    return int(num * 100) / 100;
#test
truncate(1.2345678987654)

1.23

In [15]:
trunc_node_coords = {node:(0,0) for node in node_coords} #all nodes are associated with coordinates (0,0) initially

for coords in coords_node:
    trunc_coords = (truncate(coords[0]), truncate(coords[1]))
    node = coords_node[coords]
    trunc_node_coords[node] = trunc_coords
    
print(list(trunc_node_coords.items())[55])
print(len(trunc_node_coords))

('1456859333', (12.39, 45.42))
99845


In [32]:
trunc_coords_node = {trunc_node_coords[node]:node for node in trunc_node_coords} #duplicate autoremoval
print(len(trunc_coords_node))
print(list(trunc_coords_node.items())[25])

2341
((12.2, 45.44), '2086870796')


In [42]:
#keep track of duplicates for quick substitution
dupl = {} #key = node replaced | value = node to replace it with
for tr_coords in trunc_coords_node:
    for coords in coords_node:
        if (truncate(coords[0]), truncate(coords[1])) == tr_coords:
            dupl.update({coords_node[coords]:trunc_coords_node[tr_coords]})

In [56]:
#write into a json file

with open("nodes_small.json", "w") as f:
    json.dump(trunc_node_coords, f)

In [61]:
print(roads[4])
print(type(roads[4]))

{'p1': '729936659', 'p2': '729936660', 'directed': True, 'sidewalks': [False, False], 'distance': 17.49509788424825}
<class 'dict'>


In [71]:
#change the roads
new_roads = []
for road in roads:
    road['p1'] = dupl[road['p1']]
    road['p2'] = dupl[road['p2']]
    if road['p1'] != road['p2']:
        new_roads.append(road)

In [72]:
len(new_roads)

10325

In [73]:
with open("roads_small.json", "w") as f:
    json.dump(new_roads, f)
    

In [76]:
#check!
for road in new_roads:
    if road['p1'] not in trunc_node_coords or road['p2'] not in trunc_node_coords:
        print("ERROR")